In [36]:
import random

NUM = 100 # 每次生成100道题
TYPE_DICT = ['mul', 'div']

MUL1_MIN = 20
MUL1_MAX = 300
MUL2_MIN = 4
MUL2_MAX = 9

DIV1_MIN = 100
DIV1_MAX = 1199
DIV2_MIN = 3
DIV2_MAX = 9

PROB_TYPE = ['md_rnd','five','squre']

NULL_4 = '\u0000' * 4
NULL_8 = '\u0000' * 8


In [37]:
type_rnd = random.randint(0, len(TYPE_DICT))
mul1 = random.randint(MUL1_MIN, MUL1_MAX)
mul2 = random.randint(MUL2_MIN, MUL2_MAX)

mul_res = mul1 * mul2

div1 = random.randint(DIV1_MIN, DIV1_MAX)
div2 = random.randint(DIV2_MIN, DIV2_MAX)

div_res = div1//div2

div_rem = div1 % div2

print(f"mul1 = {mul1}, mul2 = {mul2} mul_res = {mul_res}")
print(f"div1 = {div1}, div2 = {div2} div_res = {div_res} div_rem ={div_rem}")


mul1 = 197, mul2 = 8 mul_res = 1576
div1 = 483, div2 = 9 div_res = 53 div_rem =6


In [38]:
def mul_problem_gen (index:int, mul1:int, mul2:int, debug: bool = False):
    mul_res = mul1 * mul2
    format_str = f"{index:<4} | {mul1:<4} x {mul2:>4} = {NULL_4} | {NULL_4} 答案：{mul_res}"
    if debug: print(format_str)
    return format_str
    

In [39]:
mul_problem_gen(1,18,3, True)
mul_problem_gen(122,55,5, True)

1    | 18   x    3 =      |      答案：54
122  | 55   x    5 =      |      答案：275


'122  | 55   x    5 = \x00\x00\x00\x00 | \x00\x00\x00\x00 答案：275'

In [40]:
def div_problem_gen (index:int, div1:int, div2:int,  debug: bool = False):

    div_res = div1//div2

    div_rem = div1 % div2
    
    ans_str = f"{div_res}" if div_rem == 0 else f"{div_res}...{div_rem}" 
    
    format_str = f"{index:<4} | {div1:<4} \u00F7 {div2:>4} = {NULL_4} | {NULL_4} 答案：{ans_str}"
    if debug: print(format_str)
    return format_str

In [41]:
div_problem_gen(1,18,3, True)
div_problem_gen(12,55,15, True)

1    | 18   ÷    3 =      |      答案：6
12   | 55   ÷   15 =      |      答案：3...10


'12   | 55   ÷   15 = \x00\x00\x00\x00 | \x00\x00\x00\x00 答案：3...10'

In [42]:
def problem_gen (type_: str, index: int):
    problem_str = None
    if type_ == 'mul':
        mul1 = random.randint(MUL1_MIN, MUL1_MAX)
        mul2 = random.randint(MUL2_MIN, MUL2_MAX)
        problem_str = mul_problem_gen(index, mul1, mul2)
    elif type_ == 'div':
        div1 = random.randint(DIV1_MIN, DIV1_MAX)
        div2 = random.randint(DIV2_MIN, DIV2_MAX)
        problem_str = div_problem_gen(index, div1, div2)
    return problem_str
        

In [43]:
def problem_gen_five (type_: str, index: int):
    problem_str = None
    mul1 = random.randint(2, 19)*10
    mul2 = random.randint(MUL2_MIN, MUL2_MAX)*random.randint(1, 9)*20
    product = mul1 * mul2
    if type_ == 'mul':
        # mul1 = random.randint(2, 19)*50
        # mul2 = random.randint(MUL2_MIN, MUL2_MAX)*random.randint(1, 9)*2
        problem_str = mul_problem_gen(index, mul1, mul2)
    elif type_ == 'div':
        # div1 = random.randint(DIV1_MIN, DIV1_MAX)
        # div2 = random.randint(DIV2_MIN, DIV2_MAX)
        problem_str = div_problem_gen(index, product, mul2)
    return problem_str

In [44]:
def problem_gen_squre (type_: str, index: int):
    problem_str = None
    mul = random.randint(9, 20)
    product = mul * mul
    if type_ == 'mul':
        problem_str = mul_problem_gen(index, mul, mul)
    elif type_ == 'div':
        problem_str = div_problem_gen(index, product, mul)
    return problem_str

In [45]:
def problem_gen_mix(algo_type_: str, index: int, problem_type:str):
    if problem_type == 'md_rnd':
        print_str = problem_gen(algo_type_, index)
    elif problem_type == 'five':
        print_str = problem_gen_five(algo_type_, index)
    elif problem_type == 'squre':
        print_str = problem_gen_squre(algo_type_, index)
    elif problem_type == 'mix':
        single_problem_type = PROB_TYPE[random.randint(1, len(PROB_TYPE))-1]
        print_str = problem_gen_mix(algo_type_, index, single_problem_type)
    return print_str


In [46]:
def print_problem_single(round_:int, special_type:str='md_rnd'):
    all_out = ''
    for i in range (round_):
        algo_type_ = TYPE_DICT[random.randint(0, len(TYPE_DICT)-1)]
        if special_type == 'md_rnd':
            print_str = problem_gen(algo_type_, i+1)
        elif special_type == 'five':
            print_str = problem_gen_five(algo_type_, i+1)
        elif special_type == 'squre':
            print_str = problem_gen_squre(algo_type_, i+1)
        elif special_type == 'mix':
            print_str = problem_gen_mix(algo_type_, i+1, special_type)

        out = f"{print_str:<50}\n"
        
        print(out)

        all_out += out

    return all_out   


In [47]:
def print_problem_daily(days:int, nums:int, special_type:str='md_rnd'):
    for d in range(days):
        print("日期_______， 正确数目_______, 错误数目_______, 得分_______")
        print_problem_single(nums, special_type)
        print("\n"*2)

In [48]:
#print_problem_daily(10, 35)

In [49]:
def gen_problem_per_day(nums:int, special_type:str='md_rnd'):
    out=''
    out+=f"日期_____， 正确数目_____, 错误数目_____, 得分______, 类型{special_type}\n"
    out+=print_problem_single(nums, special_type)
    out+="\n"*2
    return out

In [50]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdf_name = "multi_page_problem.pdf"

def print_problem_pdf(days, nums):
    pdfmetrics.registerFont(TTFont('SimSun', './SimSun.ttf'))  #注册字体
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(fontName='SimSun', name='Song', leading=15, fontSize=13))  #leading=spacing
      
    doc = SimpleDocTemplate(pdf_name, pagesize=letter)
    
    # styles.add(ParagraphStyle(name='Chinese', fontName='AR PL UMing CN',    fontSize=20))
    styles.add(ParagraphStyle(name='Address', fontName='Arial'))
    flowables = []

    for d in range(days):
        # 将换行符替换为LineBreak对象
        # problem_text = gen_problem_per_day(nums)
        problem_text = gen_problem_per_day(nums, 'md_rnd')
        lines = problem_text.split('\n')
        for line in lines:
            flowables.append(Paragraph(line, styles['Song']))

        # 添加额外的间距
        flowables.append(Spacer(1, 12))

        if d < days - 1:  # 除了最后一页外，其他每页末尾添加分页符
            flowables.append(PageBreak())

    doc.build(flowables)

# 使用函数生成多页PDF
days = 14  # 
nums = 40  # 
print_problem_pdf(days, nums)

1    | 125  x    7 =      |      答案：875           

2    | 268  ÷    8 =      |      答案：33...4        

3    | 227  x    5 =      |      答案：1135          

4    | 180  x    6 =      |      答案：1080          

5    | 286  x    4 =      |      答案：1144          

6    | 274  x    4 =      |      答案：1096          

7    | 298  x    6 =      |      答案：1788          

8    | 62   x    4 =      |      答案：248           

9    | 589  ÷    8 =      |      答案：73...5        

10   | 41   x    6 =      |      答案：246           

11   | 21   x    5 =      |      答案：105           

12   | 73   x    5 =      |      答案：365           

13   | 747  ÷    3 =      |      答案：249           

14   | 218  ÷    5 =      |      答案：43...3        

15   | 81   x    4 =      |      答案：324           

16   | 119  x    8 =      |      答案：952           

17   | 142  x    8 =      |      答案：1136          

18   | 470  ÷    6 =      |      答案：78...2        

19   | 330  ÷    7 =      |      答案：47...1        

20   | 252  